In [1]:
# general imports

import io
import sys
import subprocess 

import numpy as np
import pyvista as pv

import galois
import json

# make sure the user has at least Python 3.7, which we need for some capture_output below
assert sys.version_info.major == 3, "This requires at least Python 3.7"
assert sys.version_info.minor >= 7, "This requires at least Python 3.7"

In [2]:
# finite field arithmetic tables and 3-D dot product calculation


# addition and multiplication tables for F_q

def make_GFq_arith_table(q):
    GFq_add = []
    GFq_mul = []
    GF = galois.GF(q)
    for i in range(q):
        x_vec = []
        y_vec = []
        for j in range(q):
            x_vec.append(i)
            y_vec.append(j)
        x = GF(x_vec)
        y = GF(y_vec)
        GFq_add.append(json.loads(str(x+y)))
        GFq_mul.append(json.loads(str(x*y)))
    return GFq_add, GFq_mul
    
def dot_product_3d(v,w,q):
    s=[]
    for i in range(3):
        s.append(GFq_mul[v[i]][w[i]])
    d = GFq_add[GFq_add[s[0]][s[1]]][s[2]]
    return d


In [3]:
# graph: generate vertices and incidence matrix
# also important vertex subsets L (the initial quadric), W (all quadrics except the initial one), C, V1 and V2
# also the center elements, given L

def generate_vertices(q):
# this generates the q^2+q+1 vertices of ER_q
# vertex_value gives the left-normalized vectors of ER_q, in lexicographic order
# you can permute vertex_value to give another ordering of the left-normalized vectors of ER_q
# vertices values as vectors in {F_q}^3, in lexicographic order

    vertex_value = []
    for i in range(int(q**3)):
        new_point = []
        new_point.insert(0, int(i%q))
        new_point.insert(0, int(((i-new_point[0])/q)%q))
        new_point.insert(0, int(((i-new_point[0]*q-new_point[1])/q**2)%q))
        if (new_point[0]==1):
            vertex_value.append(new_point)
        elif (new_point[0]==0 and new_point[1]==1):
            vertex_value.append(new_point)
        elif (new_point[0]==0 and new_point[1]==0 and new_point[2]==1):
            vertex_value.append(new_point)            
    return vertex_value

def incidence_matrix(vertices_values):
    inc_matrix = []
    num_vertices = len(vertices_values)
    for i in range(num_vertices):
        row=[]
        for j in range(num_vertices):
            row.append(0)
        inc_matrix.append(row)
    for i in range(num_vertices):
        for j in range(i,num_vertices):
            d = dot_product_3d(vertices_values[i], vertices_values[j],q)
            if d==0:
                inc_matrix[i][j]=1
                inc_matrix[j][i]=1
    return inc_matrix

def generate_W(inc_matrix):
    W=[]
    num_vertices = len(inc_matrix[0])
    for i in range(num_vertices):
        if inc_matrix[i][i]==1:
            W.append(i)
    return W

def generate_V1(L, W, inc_matrix):
    C=[]
    V1=[]
    num_vertices = len(inc_matrix[0])
    for i in range(num_vertices):
        if (i not in L) and (i not in W) and inc_matrix[i][L[0]]==1:
            C.append(i)     
    for i in range(num_vertices):
        if (i not in L) and (i not in W) and (i not in C):
            for j in W:
                if inc_matrix[i][j]==1:
                    V1.append(i)
                    break
    return (C,V1)

def generate_V2(L, W, C, V1, num_vertices):
    V2=[]
    for i in range(num_vertices):
        if (i not in L) and (i not in W) and (i not in C) and (i not in V1):
            V2.append(i)
    return V2

def generate_quad_0(W):
    # grab the last element of W to be the chosen quadric
    L=[]
    L.append(W[0])
    W.pop(0)
#     L.append(W[len(W)-1])
#     W.pop(len(W)-1)
    return (L,W)

def center_pointers(inc_matrix, l):
# gives the cluster vertices defined by a quadric
    # choose a center element, must be perp to l but not self-perp, i.e., not in W
    centerpointers = []
    for i in range(len(inc_matrix[0])):
        if inc_matrix[l][i]==1 and inc_matrix[i][i]==0:
            centerpointers.append(i)
    return centerpointers

def pos_from_pointers(positions, pointers):
    out_positions = []
    for i in range(len(pointers)):
        out_positions.append(positions[pointers[i]])
    return out_positions

def get_edges(inc_matrix, pointer_set):
# get all of the edges from a given pointer_set
    clusteredges = []
    for i in range(len(pointer_set)):
        for j in range(i+1,len(pointer_set)):
            if inc_matrix[pointer_set[i]][pointer_set[j]]==1:
                clusteredges.append([2,pointer_set[i],pointer_set[j]])
    return clusteredges

def between_ptrsets_edges(inc_matrix, pointer_set_0, pointer_set_1):
# get all of the edges between two pointer_sets
    clusteredges = []
    for i in range(len(pointer_set_0)):
        for j in range(len(pointer_set_1)):
            if inc_matrix[pointer_set_0[i]][pointer_set_1[j]]==1:
                clusteredges.append([2,pointer_set_0[i],pointer_set_1[j]])
    return clusteredges

def get_cluster(inc_matrix, vpos, l, c):
    clusterpointers = cluster_pointers(inc_matrix, l, c)
    clustervpos = pos_from_pointers(vpos, clusterpointers)
    clusteredges = get_edges(inc_matrix, clusterpointers) 
    return (clusterpointers, clustervpos, clusteredges)

def cluster_pointers(inc_matrix, l, c):
# gives pointers to the cluster vertices defined by a quadric and a center elt
    clusterpointers=[c]
    for i in range(len(inc_matrix[0])):
        if inc_matrix[c][i]==1 and inc_matrix[i][i]==0:
            clusterpointers.append(i)
    return clusterpointers

def append_edges(edges, pos, opacity, color, line_width):
    edge_sets.append(edges)    
    vpos_sets.append(pos)
    edge_opacities.append(opacity)
    edge_colors.append(color)
    line_widths.append(line_width)



In [4]:
# viz stuff

# Let's define some colors
RED = '#FF0000'
YELLOW = '#CCCC00'
GREEN = '#00DD00'
CYAN = '#00FFFF'
BLUE = '#0000FF'
MAGENTA = '#CC00CC'

LT_RED = '#FF3333'
DK_RED = '#770000'
ORANGE = '#FF8400'
LT_GREEN = '#44FF44'
DK_GREEN = '#009900'
NEON_GREEN = '#00FE94'
LT_BLUE = '#0088FF'
NEON_BLUE = '#7DF9FF'
LIGHT_COLOR = 'white'
BACKGROUND_COLOR = 'white'
GREY = '#AAAAAA'
LT_GREY = '#CCCCCC'

L_color = RED
W_color = DK_RED
C_color = LT_GREEN
V1_color = DK_GREEN
V2_color = LT_BLUE



vertex_size = 25
vertex_size_smaller = vertex_size

# these next are for the lex display if desired
# vertex_size = 70
# W_color = RED
# C_color = GREEN
# V1_color = GREEN


def visualize_graph(vertex_value, L, W, C, V1, V2, edge_sets, vpos_sets, edge_opacities, edge_colors, line_widths, fname, camera_pos, camera_rot , show_labels):
    # camera_pos views: xy, xz, yz, yx, zx, zy, iso, can also define a camera_pos = (x,y,z) tuple
#     camera_pos = 'zx'
#     camera_rot = [0,5,15]
#     # overhead view
#     camera_pos = 'xy'
#     camera_rot = [0,0,180]
    pl = pv.Plotter(lighting=None, window_size=(2000, 2000))
    light1 = pv.Light(position=(10, 10., -5.0),
                       focal_point=(0, 0, 0),
                       color=LIGHT_COLOR,  # Color temp. 5400 K
                       intensity=1.5)
    light2 = pv.Light(position=(-10, 10.0, -5.0),
                       focal_point=(0, 0, 0),
                       color=LIGHT_COLOR,  # Color temp. 2850 K
                       intensity=0.75)
#     pl.add_light(light1)  # Lighting effect to be cast onto the object
#     pl.add_light(light2)  # Second Lighting effect to be cast onto the object
    hlight = pv.Light(light_type='headlight')
    pl.add_light(hlight)  # Second Lighting effect to be cast onto the object
    pl.set_background(BACKGROUND_COLOR)  # Set the background 
            
    # pdata is the basic vertex data positions
    pdata = pv.PolyData(vpos_sets[0]) 
    
# this adds point labels of vector values -- but placed on top of the points
    if (show_labels):
        point_labels = []
        for i in range(len(vertex_value)):
            point_labels.append(vertex_value[i])        
        pl.add_point_labels(pdata, point_labels, italic=False, bold=True, font_size=30,
                            point_color='black', point_size=0, text_color='black', 
                            shape=None, 
                            render_points_as_spheres=True,
                            always_visible=True, shadow=False)  

# this adds the other edge sets    
    other_pdata = []
    for i in range(len(edge_sets)):
        other_pdata.append(pv.PolyData(vpos_sets[0]))
        other_pdata[i].lines = edge_sets[i]
        pl.add_mesh(other_pdata[i],
                    color = edge_colors[i],
                    opacity = edge_opacities[i],
                    point_size=0,
                    line_width = line_widths[i],
                    render_points_as_spheres=True
        )
        
    L_vertex_position=[]
    for i in L:
        L_vertex_position.append(vpos_sets[0][L[0]])
    L_cloud = pv.PolyData(L_vertex_position)
    pl.add_mesh(L_cloud, 
                color=L_color, 
                point_size=vertex_size,
                render_points_as_spheres=True
               )
    W_vertex_position=[]
    for i in W:
        W_vertex_position.append(vpos_sets[0][i])
    W_cloud = pv.PolyData(W_vertex_position)
    pl.add_mesh(W_cloud, 
                color=W_color, 
                point_size=vertex_size,
                render_points_as_spheres=True
               )
    C_vertex_position=[]
    for i in C:
        C_vertex_position.append(vpos_sets[0][i])
    C_cloud = pv.PolyData(C_vertex_position)
    pl.add_mesh(C_cloud, 
                color=C_color, 
                point_size=vertex_size,
                render_points_as_spheres=True
               )
    V1_vertex_position=[]
    for i in V1:
        V1_vertex_position.append(vpos_sets[0][i])
    V1_cloud = pv.PolyData(V1_vertex_position)
    pl.add_mesh(V1_cloud, 
                color=V1_color, 
                point_size=vertex_size_smaller,
                render_points_as_spheres=True
               )
    V2_vertex_position=[]
    for i in V2:
        V2_vertex_position.append(vpos_sets[0][i])
    V2_cloud = pv.PolyData(V2_vertex_position)
    pl.add_mesh(V2_cloud, 
                color=V2_color, 
                point_size=vertex_size_smaller,
                render_points_as_spheres=True
               )
    
    # Views: xy, xz, yz, yx, zx, zy, iso 
    pl.camera_position = camera_pos
    pl.camera.roll = camera_rot[0]
    pl.camera.azimuth = camera_rot[1]
    pl.camera.elevation = camera_rot[2]
    pl.camera.zoom(1)
   
    # Save the plot as a screenshot.  We can change the output resolution if desired
    pl.screenshot(fname, window_size=[2000,2000])
    #pl.add_point_labels(vpos_sets[0], range(len(vpos_sets[0])), font_size=100)
    #pl.show_axes()
    pl.show()
    



In [5]:
# graph layout: positioning vertices
# edges automatically follow

def init_pos(num_vertices):
# vertex_position gives the initial positions of q^2+q+1 points equidistant on the unit circle
# you can permute vertex_value to give another ordering of the left-normalized vectors of ER_q
# you can change vertex_position to place these elsewhere in the space, for example into 3-space
    vpos = []
    # vertices locations
    for i in range(num_vertices):
        angle_increment = 2*np.pi/num_vertices    
        vpos.append([np.sin(i*angle_increment), np.cos(i*angle_increment), 0])
    return vpos

def reset_pos(vertex_position):
    vpos = []
    for i in range(len(vertex_position)):
        vpos.append(vertex_position[i])
    return vpos

def graph_to_clusters_start(q, vertex_position, inc_matrix, L, W, C, V1, V2):
# this permutes the positions of the vertices in a way that helps to show how clusters are made
    num_vertices = len(vertex_position)
    vpos = []
    for i in range(len(vertex_position)):
        vpos.append(vertex_position[i])
    # place the element of L
    vpos[L[0]] = vertex_position[num_vertices-1-int((q-1)/2)]
    # place the elements of C to range over the top of the circle
    end_C = int((q-1)/2)
    for i in range(len(C)):
        vpos[C[i]] = vertex_position[(i-end_C)%num_vertices]
    current_pos = end_C+1
    # move the cluster elements of V1 to points corresponding to their cluster centers from C
    # want them to be more or less directly below their center
    # this loop looks at the ith element of C defining cluster C_i
    for i in range(len(C)): 
        current_pos+=1
        V1_Ci=[]
        V2_Ci=[]
        # collect the elements of V1 and V2 that are in C_i
        for j in range(len(V1)):
            if inc_matrix[V1[j]][C[len(C)-1-i]]==1:
                V1_Ci.append(V1[j])
            if inc_matrix[V2[j]][C[len(C)-1-i]]==1:
                V2_Ci.append(V2[j])
        # make triangles
        #current_pos = end_C+1 
        if (q%4 != 1):
            for j in range(int((q-1)/2)):
                # take the jth V1_temp and match it up with the V2 that is orthogonal
                # here's the jth V1
                vpos[V1_Ci[j]] = vertex_position[current_pos]
                current_pos += 1
                # take the orthogonal V2
                for k in range(len(V2_Ci)):
                    if inc_matrix[V1_Ci[j]][V2_Ci[k]]==1:
                        vpos[V2_Ci[k]] = vertex_position[current_pos]
                        current_pos += 1
                        break
        else: # q%4 == 1
            # will have (q-1)/4 pairs of V1 only tris, and (q-1)/4 pairs of V2 only tris
            for j in range(int((q-1)/4)):
                # all the tris are within V1 or within V2
                # pick the first V1 and V2 (any one not picked will do)
                vpos[V1_Ci[0]] = vertex_position[current_pos]
                current_pos += 1
                vpos[V2_Ci[0]] = vertex_position[current_pos]
                current_pos += 1
                #pick twins of V1_temp[0]
                for k in range(1,len(V1_Ci)):
                    if inc_matrix[V1_Ci[0]][V1_Ci[k]]==1:
                        vpos[V1_Ci[k]] = vertex_position[current_pos]
                        current_pos += 1
                        V1_Ci.pop(k)
                        V1_Ci.pop(0)
                        break
                #pick twins of V2_temp[0]
                for k in range(1,len(V2_Ci)):
                    if inc_matrix[V2_Ci[0]][V2_Ci[k]]==1:
                        vpos[V2_Ci[k]] = vertex_position[current_pos]
                        current_pos += 1
                        V2_Ci.pop(k)
                        V2_Ci.pop(0)
                        break                
    
    # these will be interspersed between each "rack" C_i of V1 and V2 points
    C_temp=[]
    for i in range(len(C)):
        C_temp.append(C[i])
    mid_C = int((len(C)-1)/2)
    C_temp.pop(mid_C)
    for i in range(len(W)):
        if inc_matrix[W[i]][C[mid_C]]==1:
            vpos[W[i]] = vertex_position[(end_C+1)%num_vertices]
    for j in range(len(C_temp)):
        W_temp=[]
        for i in range(len(W)):
            if inc_matrix[W[i]][C_temp[len(C_temp)-1-j]]==1:
                W_temp.append(W[i])                                
        for i in range(len(W_temp)):
            vpos[W_temp[i]] = vertex_position[(end_C+1+(j+1)*q)%num_vertices]
            
    return vpos

def graph_to_clusters_pos_2(vertex_position, L):
    # move the arbitrary quadric element in L outside and above the circle
    vpos = []
    for i in range(len(vertex_position)):
        vpos.append(vertex_position[i])
    height_L = 1.25
    vpos[L[0]] = [0.0,height_L,0.0]  
    return vpos

def graph_to_clusters_pos_3(vertex_position, W):
    # move the arbitrary quadric element in L outside and above the circle
    vpos = []
    for i in range(len(vertex_position)):
        vpos.append(vertex_position[i])
    vpos[W[1]][0] = 0
    height_W = 1.25
    dist_W = 0.5
    for i in range(len(W)):
        vpos[W[i]][0] = dist_W - i*dist_W/q
        vpos[W[i]][1] = height_W
    return vpos

def graph_layercake_pos(vertex_position, L, W, C, V1, V2):
    vpos = []
    for i in range(len(vertex_position)):
        vpos.append(vertex_position[i]) 

    num_vertices = len(vertex_position)
    vertices_angle = 2*np.pi/num_vertices
    if (q == 3%4): # if (q == 3%4), have an odd number of pairs, line the center up with the middle
        starting_angle = vertices_angle*((num_vertices-q+1)/num_vertices)
    else: # if (q == 1%4), have an even number of pairs, line the center between the two middle
        starting_angle = vertices_angle*(num_vertices-q+0.5)
    # Move the W away from the viewer and into the center
    # The W are still ordered lexicographically
    radius_W= .25
    num_W = len(W)
    angle_W = 2*np.pi/(num_W+1) 
    W_dist_up = -1.0
    starting_angle_W = 0
    vpos[L[0]][0]= radius_W*np.sin(starting_angle_W)
    vpos[L[0]][1]= radius_W*np.cos(starting_angle_W)
    vpos[L[0]][2]= W_dist_up
    
    for i in range(num_W):
        vpos[W[i]][0]= radius_W*np.sin((i+1)*angle_W+starting_angle_W)
        vpos[W[i]][1]= radius_W*np.cos((i+1)*angle_W+starting_angle_W)
        vpos[W[i]][2]= W_dist_up

#     # Permuation: this one is a nice one that works to show nice quadrics edges but only for q=7
#     # This is used in the paper

#     #save original positions out
#     pos_temp_list = []
#     for i in range(num_W):
#         pos_temp = []    
#         for j in range(3):
#             pos_temp.append(vpos[W[i]][j])
#         pos_temp_list.append(pos_temp)
#     pos_temp_list.append(vpos[L[0]])
#     # permute vertices as desired. They will drag their edges with them.
#     # this one is a nice one that works to show nice quadrics edges for q=7
#     for j in range(3):
#         vpos[W[0]][j] = pos_temp_list[7][j]
#         vpos[W[1]][j] = pos_temp_list[6][j]
#         vpos[W[2]][j] = pos_temp_list[5][j]
#         vpos[W[3]][j] = pos_temp_list[4][j]
#         vpos[W[4]][j] = pos_temp_list[2][j]
#         vpos[W[5]][j] = pos_temp_list[3][j]
#         vpos[W[6]][j] = pos_temp_list[0][j]
#         vpos[L[0]][j] = pos_temp_list[1][j]

# Move the C elements into the center
    radius_C= 1.5*radius_W
    num_C = len(C)
    angle_C = 2*np.pi/(num_C)
    #cp as in main function
    #cp = (q-1)/2
    cp = 0
    starting_angle_C = starting_angle_W+(cp+1)*angle_C/2
    starting_angle_C = starting_angle_W+(cp+0.5)*angle_C
    C_dist_up = 0
    for i in range(num_C):
        vpos[C[i]][0]= radius_C*np.sin(-(i+1)*angle_C+starting_angle_C)
        vpos[C[i]][1]= radius_C*np.cos(-(i+1)*angle_C+starting_angle_C)
        vpos[C[i]][2]= C_dist_up

# Move the V2 elements in z toward the viewer at positive z
    for i in range(len(V2)):
        vpos[V2[i]][2] += 1.0  
    return vpos



In [6]:
##### these are the prime powers q to be run
##### this algorithm is for ODD PRIME POWERS only

# Some smaller primes
#prime_power_list = [3,5,7,9,11,13,17,19,23,25,27,29,31,37,41,43,47,49,53,59,61]

# Primes of interest.
# Graphs for these primes support [556,993,2257,3783,16257] routers, having radixes [24,32,48,62,128]. 
# May want to change the opacity on edges and maybe the line width in the viz.
#prime_power_list = [23,31,47,61,127]  

prime_power_list = [3,5,7,9,11]

for q in prime_power_list:
    
    print("GF(",q,")")
    # where to save screenshots
    directory = "./ER_graphs/"
    fn_prefix = directory + "ER" + str(q)

    # the GF(q) addition and multiplication tables
    GFq_add,GFq_mul=make_GFq_arith_table(q)    
    num_vertices = q**2+q+1

    # the vertex values, in lexicographic order 
    vertex_value = generate_vertices(q)
    # the vertex positions, clockwise around the circle
    vertex_position_lex = init_pos(num_vertices)
    # the incidence matrix, calculated using the dot product over GF(q)
    inc_matrix = incidence_matrix(vertex_value)

    # all the vertex position sets that will be visualized
    vpos_sets=[]
    # all the edge sets, opacities, colors, line widths that will be visualized
    edge_sets = []
    edge_opacities = []
    edge_colors = []
    line_widths = []
    
    # generate the important vector subsets
    # L is the arbitary quadric that starts things off
    # W is the rest of the quadrics
    W = generate_W(inc_matrix)
    L,W = generate_quad_0(W)
    # C is the set of centers: the vectors that are adjacent to the element in L
    # V1 is the set of all other vectors adjacent to some quadric
    C,V1 = generate_V1(L, W, inc_matrix)
    # V2 is the rest of the vectors, not adjacent to any quadric
    V2 = generate_V2(L, W, C, V1, num_vertices)

# this is the basic layout with the edges, sets the pointers used throughout
    vpos_sets=[]
    vpos_sets.append(vertex_position_lex)
    vpointers = []
    for i in range(len(vertex_position_lex)):
        vpointers.append(i)
    edges = get_edges(inc_matrix, vpointers)
    append_edges(edges, vertex_position_lex, 1, 'black', 1)
#     append_edges(edges, vertex_position_lex, 1, 'black', 7) # lex display
    
# # visualize various layouts

    if q<17:
        edge_opacities[0] = 0.08
    elif q<27:
        edge_opacities[0] = 0.04
    elif q<47:
        edge_opacities[0] = 0.01
    elif q<61:
        edge_opacities[0] = 0.005
    else:
        edge_opacities[0] = 0.005
    line_widths[0] = 3

    # flat views
    camera_pos = 'xy'
    camera_rot = [0,0,0]
    show_labels = False
# show the initial lexicographic clockwise layout
    print("GF(",q,"): Lexicographic layout")
    visualize_graph(vertex_value, L, W, C, V1, V2, 
                    edge_sets, vpos_sets, edge_opacities, edge_colors, line_widths, 
                    fn_prefix + "_lex_cw", camera_pos, camera_rot, show_labels)
    
# this is the basic "good" layout with the edges, used throughout, do not comment out
    vertex_position_good = graph_to_clusters_start(q, vertex_position_lex, inc_matrix, L, W, C, V1, V2)
    vpos_sets=[]
    vpos_sets.append(vertex_position_good)

# # show the first graph_to_clusters layout: nicer looking than lexicographic clockwise layout
#     print("GF(",q,"): Structured layout")
#     visualize_graph(vertex_value, L, W, C, V1, V2, edge_sets, vpos_sets, edge_opacities, edge_colors, line_widths, fn_prefix + "_g2c_1", camera_pos, camera_rot, show_labels)

# cake views
    camera_pos = 'zx'
    camera_rot = [0,0,20]
    show_labels = False
# the layer cake, must start with vertex_position_good
    vertex_position_cake = graph_layercake_pos(vertex_position_good, L, W, C, V1, V2)  
    vpos_sets.append(vertex_position_cake)    # to get the interesting POV on this 3D graph, we have to play with the camera position

# # show the layer cake
#     print("GF(",q,"): Layer cake, all edges")
#     visualize_graph(vertex_value, L, W, C, V1, V2, 
#                      edge_sets, vpos_sets, edge_opacities, edge_colors, line_widths, 
#                      fn_prefix + "_cake_1cluster_3D", camera_pos, camera_rot, show_labels)

    clusterpointers = []
    # Edges: main cluster, heavy black lines
    opacity = 1.0
    color = 'black'
    linewidth = 3
    clustervptrs, clustervpos, clusteredges = get_cluster(inc_matrix, vertex_position_good, L[0], C[0])
    clusterpointers.append(clustervptrs)
    append_edges(clusteredges, clustervpos, opacity, color, linewidth)
    
    # Edges: quadric edge in RED
    opacity = 1.0
    color = RED
    linewidth = 4
    cluster_index = 0
    quad_edge = [[2, L[0], C[cluster_index]]]
    quad_pos = [vertex_position_good[L[0]], vertex_position_good[C[cluster_index]]]
    append_edges(quad_edge, quad_pos, opacity, color, linewidth)
       
    print("GF(",q,"): More structured layout, transparent edges plus dark cluster edges")
    visualize_graph(vertex_value, L, W, C, V1, V2, 
                    edge_sets, vpos_sets, edge_opacities, edge_colors, line_widths, 
                    fn_prefix + "_cake_1cluster_3D", camera_pos, camera_rot, show_labels)

    # Edges: other clusters
    #num_clusters = len(C)-1
    num_clusters = 1
    opacity = 1.0
    color = 'black'
    linewidth = 3
    for i in range(1,num_clusters+1):
        clustervptrs, clustervpos, clusteredges = get_cluster(inc_matrix, vertex_position_good, L[0], C[num_clusters-(i+1)])
        clusterpointers.append(clustervptrs)
        append_edges(clusteredges, clustervpos, opacity, color, linewidth)
        
#     print("GF(",q,"): Layer cake, transparent edges plus two clusters with dark cluster edges")
#     visualize_graph(vertex_value, L, W, C, V1, V2, 
#                     edge_sets, vpos_sets, edge_opacities, edge_colors, line_widths, 
#                     fn_prefix + "_cake_allclusters_3D", camera_pos, camera_rot, show_labels)

    print("GF(",q,"): Layer cake, overhead view, transparent edges plus some number of clusters with dark cluster edges")
    camera_pos = 'xy'
    camera_rot = [0,0,180]
    visualize_graph(vertex_value, L, W, C, V1, V2, 
                    edge_sets, vpos_sets, edge_opacities, edge_colors, line_widths, 
                    fn_prefix + "_cake_allclusters_3D", camera_pos, camera_rot, show_labels)
    # back to original camera view
    camera_pos = 'zx'
    camera_rot = [0,0,20]

    # Edges: between cluster0 to the V1 elts of cluster1 in GREEN
    if (q != 3):
    # there are none for q=3
        opacity = 1.0
        color = GREEN
        linewidth = 4
        clusterptrs_to_V1 = list(set(clusterpointers[1]) & set(V1))
        between_edges = between_ptrsets_edges(inc_matrix, clusterpointers[0], clusterptrs_to_V1)
        between_pos = pos_from_pointers(vertex_position_lex, clusterpointers[0]) + pos_from_pointers(vertex_position_lex, clusterptrs_to_V1)
        append_edges(between_edges, between_pos, opacity, color, linewidth)
    
    # Edges: between cluster0 to the V2 elts of cluster1 in BLUE
    opacity = 1.0
    color = LT_BLUE
    linewidth = 4
    clusterptrs_to_V2 = list(set(clusterpointers[1]) & set(V2))
    between_edges = between_ptrsets_edges(inc_matrix, clusterpointers[0], clusterptrs_to_V2)
    between_pos = pos_from_pointers(vertex_position_lex, clusterpointers[0]) + pos_from_pointers(vertex_position_lex, clusterptrs_to_V2)
    append_edges(between_edges, between_pos, opacity, color, linewidth)
    
    # Edges: between cluster0 and W
    opacity = 1.0
    color = RED
    linewidth = 4
    Quadrics = W+L
    between_quad_ci_edges = between_ptrsets_edges(inc_matrix, clusterpointers[0], Quadrics)
    between_quad_ci_pos = Quadrics + list(set(clusterpointers[0]).intersection(set(W)))
    append_edges(between_quad_ci_edges, between_quad_ci_pos, opacity, color, linewidth)    
    
    # make the original edges completely transparent
    edge_opacities[0] = 0    
    print("GF(",q,"): Layer cake, no main edges, two clusters with dark cluster edges and grey edges joining")
    visualize_graph(vertex_value, L, W, C, V1, V2, 
                    edge_sets, vpos_sets, edge_opacities, edge_colors, line_widths, 
                    fn_prefix + "_cake_2clusters_edgesbetween_3D", camera_pos, camera_rot, show_labels)



GF( 3 )
GF( 3 ): Lexicographic layout


ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)

GF( 3 ): More structured layout, transparent edges plus dark cluster edges


ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)

GF( 3 ): Layer cake, overhead view, transparent edges plus some number of clusters with dark cluster edges


...

ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)

GF( 11 ): Layer cake, overhead view, transparent edges plus some number of clusters with dark cluster edges


ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)

GF( 11 ): Layer cake, no main edges, two clusters with dark cluster edges and grey edges joining


ViewInteractiveWidget(height=2000, layout=Layout(height='auto', width='100%'), width=2000)